# Tidy data

"Tidy data" is a term coined by Hadley Wickam to describe data structured such that each variable is a column, each observation is a row, and each type of observational unit is a table [(Wickham, 2014)](http://dx.doi.org/10.18637/jss.v059.i10).
Data following this structure is much easier to investigate.

This Jupyter notebook works through the examples in the tidy data paper in pandas, using data sets from the [tidy data Github repository](https://github.com/hadley/tidy-data).

In [ ]:
import nu
import pandas as pd

## Defining tidy data

Here is some example data.

In [ ]:
preg = pd.read_csv('data/preg-raw.csv', header=None)
preg.columns = 'person treatment_a treatment_b'.split()
preg

A different way to look at the same data.

In [ ]:
pd.pivot_table(preg, columns='person')

Here is the tidy version of the same data.

In [ ]:
pd.melt(preg, id_vars='person', var_name='treatment', value_name='result')

Again,

1. Each variable forms a column.
2. Each observation forms a row.
3. Each type of observational unit forms a table.

## Tidying messy datasets

### Column headers are values, not variable names

Here we see a table from a Pew research study.

In [ ]:
pew = pd.read_csv('data/pew-raw.csv')
pew

This can be a convenient form for data, but it is not tidy data.
Here we can tidy it by melting.
Melting is parameterized by a list of columns that are already variables, or _colvars_ for short. 
The other columns are converted into two variables: a new variable called `income` that contains repeated column headings, and another called `freq` that contains the concatenated data values.

In [ ]:
pd.melt(pew, id_vars=['religion'], var_name='income', value_name='freq').head()

#### Billboard

In [ ]:
billboard = pd.read_csv('data/billboard.csv')
billboard.iloc[:4, :10]

In [ ]:
tidy_billboard = pd.melt(
    billboard, 
    id_vars='year artist time genre track date.entered date.peaked'.split(),
    var_name='week',
    value_name='rank')
tidy_billboard['week'] = map(lambda s: int(s.lstrip('wk')), tidy_billboard['week'])
tidy_billboard['date'] = \
    pd.to_datetime(tidy_billboard['date.entered']) \
    + pd.Series(map(lambda w: pd.DateOffset(weeks=w), tidy_billboard['week']))
tidy_billboard.pop('date.entered')
tidy_billboard.pop('date.peaked')
tidy_billboard.head()

#### TB

In [ ]:
tb = pd.read_csv('data/tb.csv')
for col in 'new_sp mu fu'.split():
    tb.pop(col)
tb.rename(columns={'iso2': 'country'}, inplace=True)
molten_tb = pd.melt(tb, id_vars='country year'.split(), var_name='column', value_name='cases')
molten_tb

In [ ]:
date_codes = '''
04 0-4
514 5-14
014 0-14
1524 15-24
2534 25-34
3544 35-44
4554 45-54
5564 55-64
65 65+
'''

In [ ]:
date_code_dict = {r[0]: r[1] for r in [s.split() for s in date_codes.strip().split('\n')]}

In [ ]:
tidy_tb = molten_tb.copy()
tidy_tb['sex'] = [s[0] for s in tidy_tb['column']]
tidy_tb['age'] = [date_code_dict[s[1:]] for s in tidy_tb['column']]
tidy_tb.pop('column')
tidy_tb.loc[pd.isnull(tidy_tb['cases']), 'cases'] = 0
tidy_tb.head()

#### Weather

The formatting for this data set is completely crazy.

In [ ]:
weather = pd.read_table(
    'data/weather.tsv', 
    sep=' [ ]*I[ ]*', 
    names=['info']+['d'+str(d) for d in range(2, 33)])
splut = [s.split() for s in weather['info']]
weather['info'] = [s[0] for s in splut]
weather['d1'] = [s[1] for s in splut]
weather.head()

Check that there's nothing interesting happening in the artifactual day 32 column, then drop it.
Replace those silly -9999's with NaN.

In [ ]:
assert(np.all(pd.isnull(weather['d32'])))
weather.pop('d32')
for d in 'd29 d30'.split():
    weather.loc[weather[d] == '-9999   -9999', d] = np.nan
weather.loc[weather['d31'] == -9999, 'd31'] = np.nan
print list(weather.iloc[0])
print list(weather.iloc[31])

weather = weather[['info']+['d'+str(d) for d in range(1, 32)]]  # Reorder columns.
weather.head()

In [ ]:
weather.tail()